# Alternative PygmalionAI notebook for Google Colab

Version: GPU

This notebook is an unofficial method to run [PygmalionAI LLMs](https://github.com/PygmalionAI) that uses **oobabooga**'s [text-generation-webui project](https://github.com/oobabooga/text-generation-webui).

Run the cells below in order to set it up.

---

After step 3, if you enabled Google Drive, you may upload character definitions in `.json` format by navigating to:
  * [**Drive**](https://drive.google.com/) &rarr; **My Drive** &rarr; **Colab-TextGen** &rarr; **characters**

At the end of step 5 click on the `.gradio.live` link to access the web interface.

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

# From: https://github.com/henk717/KoboldAI

%%html
<b>Press play on the music player to keep the tab alive (Uses only 13MB of data)</b><br/>
<audio src="https://henk.tech/colabkobold/silence.m4a" controls>

In [ ]:
#@title 2. Prepare the Conda environment

%cd /tmp
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
%env PYTHONPATH=
!which conda 2>/dev/null || bash Miniconda3-latest-Linux-x86_64.sh -b -p /usr/local -f
%env CI=true
!conda init -q bash
!source /usr/local/bin/activate && conda update -y -q -c defaults --all
!source /usr/local/bin/activate && conda create -y -q -n textgen
!source /usr/local/bin/activate && conda activate textgen && conda install -y -q -c defaults conda python
!source /usr/local/bin/activate && conda activate textgen && conda install -y -q -c pytorch -c nvidia pytorch torchvision torchaudio pytorch-cuda=11.7
!source /usr/local/bin/activate && conda clean -y -q --all

In [ ]:
#@title 3. Clone the repository and install dependencies

Enable_Google_drive = True #@param {type:"boolean"}

if Enable_Google_drive:
  from google.colab import drive
  drive.mount('/content/drive')
else:
  import os
  if not os.path.exists("/content/drive"):
    os.mkdir("/content/drive")
  if not os.path.exists("/content/drive/MyDrive/"):
    os.mkdir("/content/drive/MyDrive/")

# Download Github repo
!git -C /content/drive/MyDrive/Colab-TextGen pull || git clone https://github.com/oobabooga/text-generation-webui /content/drive/MyDrive/Colab-TextGen

%cd /content/drive/MyDrive/Colab-TextGen

# Set model directories outside of Google Drive
!mountpoint -q models || mv -f models models_backup
!mountpoint -q torch-dumps || mv -f torch-dumps torch-dumps_backup
!mkdir -p models torch-dumps /content/models /content/torch-dumps
!mountpoint -q models || mount --bind /content/models models
!mountpoint -q torch-dumps || mount --bind /content/torch-dumps torch-dumps

# Install requirements
!source /usr/local/bin/activate && conda activate textgen \
  && pip install --no-cache-dir --progress-bar=off --root-user-action=ignore -r requirements.txt diffusers

In [ ]:
#@title 4. Select a PygmalionAI model

Model = "Pygmalion 6B" #@param ["Pygmalion 6B", "Pygmalion 6B experimental", "Pygmalion 2.7B", "Pygmalion 1.3B", "Pygmalion 350M"] {allow-input: false}
Reshard_model = True #@param {type: "boolean"}
Max_shard_size = "7500MB" #@param ["7500MB", "3GB", "1GB"] {allow-input: true}

if Model == "Pygmalion 6B":
  huggingface_org = "PygmalionAI"
  huggingface_repo = "pygmalion-6b"
  huggingface_branch = "main"
  model_name = "pygmalion-6b"
elif Model == "Pygmalion 6B experimental":
  huggingface_org = "PygmalionAI"
  huggingface_repo = "pygmalion-6b"
  huggingface_branch = "dev"
  model_name = "pygmalion-6b_dev"
elif Model == "Pygmalion 2.7B":
  huggingface_org = "PygmalionAI"
  huggingface_repo = "pygmalion-2.7b"
  huggingface_branch = "main"
  model_name = "pygmalion-2.7b"
elif Model == "Pygmalion 1.3B":
  huggingface_org = "PygmalionAI"
  huggingface_repo = "pygmalion-1.3b"
  huggingface_branch = "main"
  model_name = "pygmalion-1.3b"
elif Model == "Pygmalion 350M":
  huggingface_org = "PygmalionAI"
  huggingface_repo = "pygmalion-350m"
  huggingface_branch = "main"
  model_name = "pygmalion-350m"

%cd /content/drive/MyDrive/Colab-TextGen

# Download the model
![[ ! -f models/$model_name/config.json ]] && source /usr/local/bin/activate && conda activate textgen \
  && python download-model.py $huggingface_org/$huggingface_repo --branch $huggingface_branch

# Convert model if the reshard option was checked
if Reshard_model:
  original_path = model_name
  resharded_path = model_name + f'_resharded'
  launch_model = resharded_path
else:
    launch_model = model_name
![[ $Reshard_model = True ]] && source /usr/local/bin/activate && conda activate textgen \
  && wget -c https://gist.githubusercontent.com/81300/fe5b08bff1cba45296a829b9d6b0f303/raw/2bad1df5796328d7385ca7923fcd703dc0243dde/reshard-model.py \
  && python reshard-model.py --src-path models/$original_path --out-path models/$resharded_path --max-shard-size $Max_shard_size --offload /content/offload \
  && cp -fa models/$original_path/*token*.json models/$original_path/vocab.json models/$resharded_path

In [ ]:
#@title 5. Launch the web interface

%cd /content/drive/MyDrive/Colab-TextGen

!source /usr/local/bin/activate && conda activate textgen \
  && python server.py --share --cai-chat --no-stream --auto-devices --model $launch_model

In [ ]:
#@title Cleanup all models

#@markdown Free up space within the current runtime.

!find /content/models -mindepth 1 -maxdepth 1 -type d -exec rm -rv "{}" \;
!find /content/torch-dumps -mindepth 1 -exec rm -rv "{}" \;